# Задание

- Выберите набор данных (датасет) для решения задачи классификации или регресии.
- В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.
- С использованием метода train_test_split разделите выборку на обучающую и тестовую.
- Обучите следующие ансамблевые модели:
  - две модели группы бэггинга (бэггинг или случайный лес или сверхслучайные деревья);
  - AdaBoost;
  - градиентный бустинг.
- Оцените качество моделей с помощью одной из подходящих для задачи метрик.Сравните качество полученных моделей.

Выбранный датасет (алмазы):
Содержание:
- Цена в долларах США ($326--$18,823)
- Карат  (0.2--5.01)
- Качество обрезки (Низкое, Хорошее, Очень хорошее, Премиум, Идеальное)
- Цвет бриллианта, от J (худший) до D (лучший)
- Измерение прозрачности бриллианта (I1 (худшее), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (лучшее))
- X длина в мм (0--10.74)
- Y ширина в мм (0--58.9)
- Z глубина в мм (0--31.8)
- Общий процент глубины = z / среднее(x, y) = 2 * z / (x + y) (43--79)
- Ширина вершины бриллианта относительно самой широкой точки (43--95)

In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

In [42]:
data = pd.read_csv('diamonds.csv')

In [43]:
data = data.drop(columns=['id'])
data.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

In [44]:
data.shape, data.price.shape

((53940, 10), (53940,))

In [45]:
data.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [46]:
data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Кодирование категориальных признаков

In [47]:
cut_order = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}
color_order = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6}
clarity_order = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}
data['cut_encoded'] = data['cut'].map(cut_order)
data['color_encoded'] = data['color'].map(color_order)
data['clarity_encoded'] = data['clarity'].map(clarity_order)
df_encoded = data.drop(columns=['cut', 'color', 'clarity'])
print(df_encoded)

       carat  depth  table  price     x     y     z  cut_encoded  \
0       0.23   61.5   55.0    326  3.95  3.98  2.43            4   
1       0.21   59.8   61.0    326  3.89  3.84  2.31            3   
2       0.23   56.9   65.0    327  4.05  4.07  2.31            1   
3       0.29   62.4   58.0    334  4.20  4.23  2.63            3   
4       0.31   63.3   58.0    335  4.34  4.35  2.75            1   
...      ...    ...    ...    ...   ...   ...   ...          ...   
53935   0.72   60.8   57.0   2757  5.75  5.76  3.50            4   
53936   0.72   63.1   55.0   2757  5.69  5.75  3.61            1   
53937   0.70   62.8   60.0   2757  5.66  5.68  3.56            2   
53938   0.86   61.0   58.0   2757  6.15  6.12  3.74            3   
53939   0.75   62.2   55.0   2757  5.83  5.87  3.64            4   

       color_encoded  clarity_encoded  
0                  5                1  
1                  5                2  
2                  5                4  
3                  1   

### Разделим выборку на обучающую и тестовую

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
X = df_encoded.drop("price", axis=1)
y = df_encoded["price"]

In [50]:
print(X.head(), "\n")
print(y.head())

   carat  depth  table     x     y     z  cut_encoded  color_encoded  \
0   0.23   61.5   55.0  3.95  3.98  2.43            4              5   
1   0.21   59.8   61.0  3.89  3.84  2.31            3              5   
2   0.23   56.9   65.0  4.05  4.07  2.31            1              5   
3   0.29   62.4   58.0  4.20  4.23  2.63            3              1   
4   0.31   63.3   58.0  4.34  4.35  2.75            1              0   

   clarity_encoded  
0                1  
1                2  
2                4  
3                3  
4                1   

0    326
1    326
2    327
3    334
4    335
Name: price, dtype: int64


In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

In [52]:
X_train.shape, y_train.shape,

((37758, 9), (37758,))

In [53]:
X_test.shape, y_test.shape

((16182, 9), (16182,))

### Обучение моделей

In [54]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [55]:
bagging_regressor = BaggingRegressor()
bagging_regressor.fit(X_train, y_train)

BaggingRegressor()

In [56]:
random_forest_regressor = RandomForestRegressor()
random_forest_regressor.fit(X_train, y_train)

RandomForestRegressor()

In [57]:
ada_boost_regressor = AdaBoostRegressor()
ada_boost_regressor.fit(X_train, y_train)

AdaBoostRegressor()

In [58]:
gradient_boosting_regressor = GradientBoostingRegressor()
gradient_boosting_regressor.fit(X_train, y_train)

GradientBoostingRegressor()

In [59]:
bagging_pred = bagging_regressor.predict(X_test)
print(bagging_pred)

[  949.2 12782.9  8653.9 ...  1117.3  7214.3  4186.4]


In [60]:
random_forest_pred = random_forest_regressor.predict(X_test)
print(random_forest_pred)

[  916.07 12987.31  8395.15 ...  1125.82  7515.2   4205.63]


In [61]:
ada_boost_pred = ada_boost_regressor.predict(X_test)
print(ada_boost_pred)

[ 1166.18140704 13921.07287982  9199.11009174 ...  1181.67655786
  9311.3391819   3982.07225691]


In [62]:
gradient_boosting_pred = gradient_boosting_regressor.predict(X_test)
print(gradient_boosting_pred)

[ 1028.67531217 13456.96439484  9085.2257314  ...  1116.44485684
  7803.85766126  3954.69303177]


In [63]:
bagging_mse = mean_squared_error(y_test, bagging_pred)
random_forest_mse = mean_squared_error(y_test, random_forest_pred)
ada_boost_mse = mean_squared_error(y_test, ada_boost_pred)
gradient_boosting_mse = mean_squared_error(y_test, gradient_boosting_pred)

In [64]:
bagging_r2_score = r2_score(y_test, bagging_pred)
random_forest_r2_score = r2_score(y_test, random_forest_pred)
ada_boost_r2_score = r2_score(y_test, ada_boost_pred)
gradient_boosting_r2_score = r2_score(y_test, gradient_boosting_pred)

In [65]:
print("Bagging MSE:", bagging_mse)
print("Random Forest MSE:", random_forest_mse)
print("AdaBoost MSE:", ada_boost_mse)
print("Gradient Boosting MSE:", gradient_boosting_mse)

Bagging MSE: 333084.1066192747
Random Forest MSE: 305428.0759972728
AdaBoost MSE: 1320805.1052929792
Gradient Boosting MSE: 408573.30124690785


In [66]:
print("Bagging r2_score:", bagging_r2_score)
print("Random Forest r2_score:", random_forest_r2_score)
print("AdaBoost r2_score:", ada_boost_r2_score)
print("Gradient Boosting r2_score:", gradient_boosting_r2_score)

Bagging r2_score: 0.9790063963578343
Random Forest r2_score: 0.9807494988765549
AdaBoost r2_score: 0.916752380801032
Gradient Boosting r2_score: 0.9742484682556378


Видно, что максимальная точность будет у модели случайного леса - ~98%, потом у бэггинга - 97.9% и градиентного бустинга - 97.4%

## Часть 2
### Задание

- Обучите следующие ансамблевые модели:
  - одну из моделей группы стекинга.
  - модель многослойного персептрона. По желанию, вместо библиотеки scikit-learn возможно использование библиотек TensorFlow, PyTorch или других аналогичных библиотек.
  - двумя методами на выбор из семейства МГУА (один из линейных методов COMBI / MULTI + один из нелинейных методов MIA / RIA) с использованием библиотеки gmdh.
- Оцените качество моделей с помощью одной из подходящих для задачи метрик. Сравните качество полученных моделей.

Стекинг

In [67]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor

In [68]:
base_models = [
    ('linear_regression', LinearRegression()),
    ('gradient_boosting', GradientBoostingRegressor()),
    ('random_forest', RandomForestRegressor())
]

In [69]:
meta_model = LinearRegression()

In [70]:
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)


In [71]:
stacking_regressor.fit(X_train, y_train)

StackingRegressor(estimators=[('linear_regression', LinearRegression()),
                              ('gradient_boosting',
                               GradientBoostingRegressor()),
                              ('random_forest', RandomForestRegressor())],
                  final_estimator=LinearRegression())

In [72]:
stacking_pred = stacking_regressor.predict(X_test)

In [73]:
stacking_mse = mean_squared_error(y_test, stacking_pred)
print("Stacking Regressor MSE:", stacking_mse)

Stacking Regressor MSE: 299522.0644998017


In [74]:
stacking_r2_score = r2_score(y_test, stacking_pred)
print("Stacking Regressor R²:", stacking_r2_score)

Stacking Regressor R²: 0.9811217425892389


### Модель многослойного персептрона (MLP)

In [75]:
from sklearn.neural_network import MLPRegressor

In [76]:
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=1)

In [77]:
mlp_regressor.fit(X_train, y_train)

c:\Users\Ivan\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(100, 50), random_state=1)

In [78]:
mlp_pred = mlp_regressor.predict(X_test)

In [79]:
mlp_mse = mean_squared_error(y_test, mlp_pred)
print("MLP Regressor MSE:", mlp_mse)

MLP Regressor MSE: 505786.92657766235


In [80]:
mlp_r2_score = r2_score(y_test, mlp_pred)
print("MLP Regressor R²:", mlp_r2_score)

MLP Regressor R²: 0.9681212941327829


### МГУА 

In [81]:
%pip install gmdh

Note: you may need to restart the kernel to use updated packages.


C:\Users\Ivan\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
C:\Users\Ivan\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
C:\Users\Ivan\AppData\Roaming\Python\Python311\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [82]:
from gmdh import Combi, split_data

In [83]:
x_train, x_test, y_train, y_test = split_data(X, y)

In [84]:
model = Combi()
model.fit(x_train, y_train)

In [85]:
y_predicted = model.predict(x_test)
print('y_predicted: ', y_predicted)
print('y_test: ', y_test)

y_predicted:  [1855.36567744 1837.45424855 1861.71781908 ... 3036.42202441 2757.53036992
 3261.72207152]
y_test:  [1389. 1389. 1389. ... 2757. 2757. 2757.]


In [86]:
model.get_best_polynomial()

'y = 7208.074*x1 - 37.554*x2 - 25.4222*x3 + 138.8447*x7 + 281.7583*x8 + 562.4366*x9 - 1218.6159'

In [87]:
model_r2_score = r2_score(y_test, y_predicted)
print("model Regressor R²:", model_r2_score)

model Regressor R²: -1.0427839691443066


In [88]:
from gmdh import Mia

In [89]:
mia = Mia()
mia.fit(x_train, y_train)

In [90]:
y_pred = mia.predict(x_test)
print('y_predicted: ', y_pred)
print('y_test: ', y_test)

y_predicted:  [1678.85058369 1642.15104252 1790.96261059 ... 2796.58260871 3374.75311997
 2631.59266675]
y_test:  [1389. 1389. 1389. ... 2757. 2757. 2757.]


In [91]:
mia.get_best_polynomial()

'y = - 2953.5914*x4 - 769.3024*x9 + 250.8486*x4*x9 + 419.459*x4^2 - 41.8825*x9^2 + 4942.8171'

In [92]:
mia_r2_score = r2_score(y_test, y_pred)
print("model Regressor R²:", mia_r2_score)

model Regressor R²: -0.5698743410496401
